In [1]:
import sklearn.tree
import numpy as np
from StringIO import StringIO
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

# Initiate classifier
infoGain_clf = sklearn.tree.DecisionTreeClassifier(criterion='entropy')
bayes_clf = BernoulliNB()
ss = StringIO()

def load_data():

    return np.genfromtxt('house-votes-84.data', dtype=None, delimiter=',')


def process(process_type, classifier_type):

    col_names = [
        'RepubOrDemo', 'handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution',
        'physician-fee-freeze', 'el-salvador-aid', 'religious-groups-in-schools', 'anti-satellite-test-ban',
        'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 'education-spending',
        'superfund-right-to-sue', 'crime', 'duty-free-exports', 'export-administration-act-south-africa'
    ]

    my_votes_array = load_data()

    x_matrix1 = my_votes_array[:, 1:]
    x_matrix2 = my_votes_array

    feature_names = np.array(col_names[1:])

    label_name = ['Republican', 'Democrat']

    if process_type == 'ternary':
        x_matrix = ternary(x_matrix1)
        x_matrix = x_matrix.astype(int)
        y = my_votes_array[:, :1]
    elif process_type == 'skip':
        x_matrix = skip(x_matrix2)
        y = x_matrix[:, :1]
        x_matrix = x_matrix[:, 1:]
        x_matrix = x_matrix.astype(int)
    else:
        x_matrix = replace_cols_with_y_or_n(my_votes_array)
        x_matrix = skip(x_matrix)
        y = x_matrix[:, :1]
        x_matrix = x_matrix[:, 1:]
        x_matrix = x_matrix.astype(int)

    # Fit classifier
    if classifier_type is 'decision':
        x_train, x_test, y_train, y_test = train_test_split(x_matrix, y, test_size=0.2, random_state=0)
        scores = cross_val_score(infoGain_clf, x_train, y_train.ravel(), scoring='accuracy', cv=5)

        # print 'Cross Validation Score Mean on Training Set'
        # print scores.mean()

        train_and_evaluate(infoGain_clf, x_train, x_test, y_train, y_test)
    else:
        x_train, x_test, y_train, y_test = train_test_split(x_matrix, y, test_size=0.2, random_state=0)
        scores = cross_val_score(bayes_clf, x_train, y_train.ravel(), scoring='accuracy', cv=5)
        # print 'Cross Validation Score Mean on Training Set'
        # print scores.mean()

        train_and_evaluate(bayes_clf, x_train, x_test, y_train, y_test)


def train_and_evaluate(clf, x_train, x_test, y_train, y_test):
    clf.fit(x_train, y_train.ravel())

    # print 'Accuracy in training set:'
    # print clf.score(x_train, y_train)
    # print 'Accuracy on test set:'
    # print clf.score(x_test, y_test)

    y_pred = clf.predict(x_test)

    print 'Classification Report:'
    print metrics.classification_report(y_test, y_pred)

    print 'Confusion Matrix:'
    print metrics.confusion_matrix(y_test, y_pred)
    print '\n'


def ternary(data):

    data[data == 'y'] = 1
    data[data == 'n'] = 0
    data[data == '?'] = 2

    return data


def skip(data):

    data[data == 'y'] = 1
    data[data == 'n'] = 0

    indices = np.where(data == '?')
    data = np.delete(data, list(set(indices[0])), 0)

    return data


def get_tree_prediction(row_num, data, y):
    predicted = infoGain_clf.predict(data[row_num:, :])
    print "predicted:", predicted
    print "truth", y[row_num:]
    print '\n'

    return predicted


def get_bayes_prediction(row_num, data, y):
    print "predicted:", bayes_clf.predict(data[row_num:, :])
    print "truth", y[row_num:]
    print '\n'


def replace_cols_with_y_or_n(data):

    my_data = data

    for i in range(1, 17):

        yes_cols = np.where(my_data[:, i] == 'y')
        no_cols = np.where(my_data[:, i] == 'n')

        if len(yes_cols[0]) >= len(no_cols[0]):
            my_data[my_data[:, i] == '?', i] = 'y'
        elif len(yes_cols[0]) < len(no_cols[0]):
            my_data[my_data[:, i] == '?', i] = 'n'

    return my_data


print 'DECISION::TERNARY::'
process('ternary', 'decision')
print '---------------------------------------------------------------------'

print 'DECISION::SKIP::'
process('skip', 'decision')
print '---------------------------------------------------------------------'

print 'DECISION::REPLACE::'
process('replace', 'decision')
print '---------------------------------------------------------------------'
#
print 'NAIVE_BAYES::TERNARY::'
process('ternary', 'naive_bayes')
print '---------------------------------------------------------------------'
#
print 'NAIVE_BAYES::SKIP::'
process('skip', 'naive_bayes')
print '---------------------------------------------------------------------'

print 'NAIVE_BAYES::REPLACE::'
process('replace', 'naive_bayes')
print '---------------------------------------------------------------------'



DECISION::TERNARY::
Classification Report:
             precision    recall  f1-score   support

   democrat       0.98      0.94      0.96        52
 republican       0.92      0.97      0.94        35

avg / total       0.96      0.95      0.95        87

Confusion Matrix:
[[49  3]
 [ 1 34]]


---------------------------------------------------------------------
DECISION::SKIP::
Classification Report:
             precision    recall  f1-score   support

   democrat       0.88      0.96      0.92        23
 republican       0.95      0.88      0.91        24

avg / total       0.92      0.91      0.91        47

Confusion Matrix:
[[22  1]
 [ 3 21]]


---------------------------------------------------------------------
DECISION::REPLACE::
Classification Report:
             precision    recall  f1-score   support

   democrat       0.96      0.94      0.95        52
 republican       0.92      0.94      0.93        35

avg / total       0.94      0.94      0.94        87

Confusion M